### Solving Aiyagari (1994) model
#### 07/2019

#### Calibrating the Markov process


We need to calibrate the markov process so that:

\begin{equation}
\begin{bmatrix}
e & u
\end{bmatrix}
\begin{bmatrix}
\pi_{ee} & \pi_{eu} \\
\pi_{ue} & \pi_{uu}
\end{bmatrix} = \begin{bmatrix}
e & u
\end{bmatrix}
\end{equation}

with $e = 0.9 \text{ and } \pi_{ee} = 0.9$

We get that:
\begin{equation}
\begin{bmatrix}
\pi_{ee} & \pi_{eu} \\
\pi_{ue} & \pi_{uu}
\end{bmatrix} = \begin{bmatrix}
0.9 & 0.1 \\
0.9 & 0.1
\end{bmatrix}
\end{equation}

In [15]:
import numpy as np
from scipy import interpolate
import numba as nb

#Economic parameters
Π = np.array([[0.9,0.1], [0.9, 0.1]] )
Uss = 0.1
α = 0.36
β = 0.96
δ = 0.06



# Basic functions

# Return on capital
r_f = lambda k,u: α*k**(α - 1) * (1 - u)** (1 - α )


# Return on labor
w_f = lambda k,u: (1-α)*k**(α ) * (1 - u)** (- α )

#Aggregate labor supply
L_f = lambda u: 1 - u

#Utility
def u(c):
    if c > 0:
        return np.log(c)
    else:
        return -500000
@nb.njit
# Interpolant to evaluate Value Function
def interpolant(x,Kgrid,Vp):
    t = interpolate.interp1d(Kgrid,Vp,fill_value="extrapolate")
    return t(x)

@nb.njit
def Bellman(Kp, *args):
    R, W, Kgrid, Vp, Ε, j, l, Π = args
    
    Today = u(R* Kgrid[j] + W*Ε[l] + (1 - δ) * Kgrid[j] - Kp)
#     print("Value for Today", Today)
    exp= 0
    for i in range(Nϵ):
        exp +=  Π[l,i]*interpolant(Kp,Kgrid,Vp[:,i])  
    
    Value = Today + β * exp

    return - Value




#### Compute the steady state equilibrium for K

In [6]:
from scipy.optimize import minimize



# Computational parameters
N = 50
Nϵ = 2
epsi = 1
epsi2 = 1
tol = 1e-6
maxiter = 500
simuT = 10000


# Make grid
Klow   = 0.001
Khigh  = 40
Kgrid = np.linspace(Klow, Khigh, N)

#Steady State capital
Kstar = ( (1/β - 1+δ) /(α*(1-Uss)**(1-α))) ** (1 / (α - 1))
Ustar = Uss
Ε = np.array([1,0])


iter  = 0
iter2 = 0
#Vp=np.zeros([N,Nϵ])
Kp=np.zeros([N,Nϵ])
Vp_new=np.zeros([N,Nϵ])
Kp_new=np.zeros([N,Nϵ])



while (epsi2 > tol) & (iter2 < maxiter):
    epsi = 1
    iter = 0
    # Dynamic Programming Step
    ##########################
    
    while (epsi > tol) & (iter < maxiter):
    
        for j in range(N):

            for l in range(Nϵ):

                R = r_f(Kstar, Ustar)
                W = w_f(Kstar, Ustar)

                params = R, W, Kgrid, Vp, Ε, j, l, Π
                solve = minimize(Bellman,Kp[j,l], args=(params))
                Kp_new[j,l] = solve.x
                Vp_new[j,l] = - solve.fun


        epsi = ((Vp_new - Vp) ** 2).sum()
        print("Error term for the Value Function: ", epsi)
        #Update
        Kp=Kp_new*1.
        Vp = Vp_new*1.
        iter=iter+1
        
    # Simulation step
    #################
    
    np.random.seed(seed=13)
    ϵ = np.random.binomial(1, 0.1, simuT)
    Ksimu = np.zeros([simuT])
    #Usimu = np.zeros([simuT])
    
    Ksimu[0] = Kstar
    @nb.njit
    for t in range(1,simuT):
        print(Ksimu[t-1])
        Ksimu[t] = interpolant(Ksimu[t-1], Kgrid, Kp[:,ϵ[t]])
    
    Kstar_new = np.mean(Ksimu[500:])
    print("New value for Kstar", Kstar_new)
    epsi2 = (Kstar_new - Kstar) ** 2
    print("Error term for K*: ", epsi)
    Kstar = Kstar_new
        
    iter2=iter2+1


    

Error term for the Value Function:  9.158045806707913e-07
New value for Kstar -1.862349010033819e+180
Error term for K*:  9.158045806707913e-07


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:67: RuntimeWarning: overflow encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in greater


Error term for the Value Function:  7.222753343749064e+16
Error term for the Value Function:  9.24669097454193e+33


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in greater
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in greater
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in greater
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in grea

Error term for the Value Function:  1.157306673501186e+51
Error term for the Value Function:  1.4381944846357175e+68


...

Error term for the Value Function:  inf


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in greater
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:613: RuntimeWarning: overflow encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in add
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:425: RuntimeWarning: invalid value encountered in greater
  if (phi_a1 > phi0 + c1 * alpha1 * derphi0) or \
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:426: RuntimeWarning: invalid value encountered in greater_equal
  ((phi_a1 >= phi_a0) and (i > 1)):
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in greater
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:613: RuntimeWarning: overflow enc

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:391: RuntimeWarning: invalid value encountered in true_divide
  alpha1 = min(1.0, 1.01*2*(phi0 - old_phi0)/derphi0)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:391: RuntimeWarning: invalid value encountered in less
  alpha1 = min(1.0, 1.01*2*(phi0 - old_phi0)/derphi0)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in greater
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:613: RuntimeWarning: overflow encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in add
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:425: RuntimeWarning: invalid value encountered in greater
  if (phi_a1 > phi0 + c1 * alpha1 * derphi0) or \
C:\ProgramData\Anaconda3\lib\site-

Error term for the Value Function:  nan


KeyboardInterrupt: 

In [ ]:
print(Π[1,0])

In [ ]:
print(Kstar)

In [7]:
print(Ksimu)

[-1.86234901e+180              inf             -inf ...  0.00000000e+000
  0.00000000e+000  0.00000000e+000]


In [13]:
Vp=np.zeros([N,Nϵ])
Kp=np.zeros([N,Nϵ])
Vp_new=np.zeros([N,Nϵ])
Kp_new=np.zeros([N,Nϵ])
Kstar = 6.7
epsi = 1
iter = 0
while (epsi > tol) & (iter < maxiter):
    
        for j in range(N):

            for l in range(Nϵ):

                R = r_f(Kstar, Ustar)
                W = w_f(Kstar, Ustar)

                params = R, W, Kgrid, Vp, Ε, j, l, Π
                solve = minimize(Bellman,Kp[j,l], args=(params))
                Kp_new[j,l] = solve.x
                Vp_new[j,l] = - solve.fun


        epsi = ((Vp_new - Vp) ** 2).sum()
        print("Error term for the Value Function: ", epsi)
        #Update
        Kp=Kp_new*1.
        Vp = Vp_new*1.
        iter=iter+1
        
        # Simulation step
        #################

        np.random.seed(seed=13)
        ϵ = np.random.binomial(1, 0.1, simuT)
        Ksimu = np.zeros([simuT])
        #Usimu = np.zeros([simuT])

        Ksimu[0] = Kstar
        for t in range(1,simuT):
            for i in range(Nϵ):
                Ksimu[t] = interpolant(Ksimu[t-1], Kgrid, Kp[:,ϵ[t]])

        Kstar_new = np.mean(Ksimu[500:])

Error term for the Value Function:  14000.636233827388


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:75: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)


Error term for the Value Function:  623.8125504086208
Error term for the Value Function:  207.96899223449574
Error term for the Value Function:  140.1138348136835
Error term for the Value Function:  122.36752375286666
Error term for the Value Function:  113.03182892777457
Error term for the Value Function:  104.51662019855156
Error term for the Value Function:  96.65581504183281
Error term for the Value Function:  89.39863192989179
Error term for the Value Function:  82.6982497784653
Error term for the Value Function:  76.5114983232204
Error term for the Value Function:  70.79857269333567
Error term for the Value Function:  65.52277031340284
Error term for the Value Function:  60.65024842256592
Error term for the Value Function:  56.14980057841339
Error term for the Value Function:  51.99265062514781
Error term for the Value Function:  48.152262818704635
Error term for the Value Function:  44.6041667808448
Error term for the Value Function:  41.325796197723164
Error term for the Value 

Error term for the Value Function:  0.21621473204091318
Error term for the Value Function:  0.2153344535187967
Error term for the Value Function:  0.21449479809644462
Error term for the Value Function:  0.21369398490382246
Error term for the Value Function:  0.2129303159978517
Error term for the Value Function:  0.21220217491628096
Error term for the Value Function:  0.21150800681535398
Error term for the Value Function:  0.21084633400856456
Error term for the Value Function:  0.21021575451309987
Error term for the Value Function:  0.20961490437080849
Error term for the Value Function:  0.20904250615882727
Error term for the Value Function:  0.20849734959510308
Error term for the Value Function:  0.2079783018073956
Error term for the Value Function:  0.20748422576480766
Error term for the Value Function:  0.20701403832799015
Error term for the Value Function:  0.20656670863360463
Error term for the Value Function:  0.20614128501662193
Error term for the Value Function:  0.2057368408462

KeyboardInterrupt: 